In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
#laptop
#df_DropSeq_MCF7 = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
MCF_DropSeq = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt",
                  delimiter="\ ", engine='python', index_col=0)
MCF_DropSeq = MCF_DropSeq.T

In [ ]:
#laptop
#df = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
HCC_DropSeq = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
HCC_DropSeq = HCC_DropSeq.T

In [ ]:
MCF_SS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\MCF7_SmartS_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)
HCC_SS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\HCC1806_SmartS_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)

In [ ]:
#function to add new 'Oxia' row to the data frame
#1 is Normoxia, 0 is Hypoxia
def oxia(df):
    new_row = {col: None for col in df.columns}
    for col in df.columns:
        if '_Normoxia' in col:
            new_row[col] = 1
        elif '_Hypoxia' in col:
            new_row[col] = 0
    df = pd.concat([df, pd.DataFrame(new_row, index=['oxia'])])
    return df

In [ ]:
#add all files to a single dictionary which will be worked on
data = {'MCF_DropSeq': MCF_DropSeq, 'HCC_DropSeq': HCC_DropSeq, 'MCF_SS': MCF_SS, 'HCC_SS': HCC_SS}

In [ ]:
#apply oxia function to all dataframes
for key in data:
    data[key] = oxia(data[key])

In [ ]:
#function to split the data into X and y
def split_data(df):
    X = df.drop('oxia', axis=0)
    y = df.loc['oxia']
    return (X, y)

In [ ]:
#split all dataframes into X and y
for key in data:
    data[key] = split_data(data[key])

In [ ]:
#function to split the data into training and testing data
def split_train_test(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return (X_train, X_test, y_train, y_test)

In [ ]:
#split all dataframes into training and testing data
for key in data:
    data[key] = split_train_test(data[key][0], data[key][1])

In [ ]:
#dictionary of models:
models = {'Logistic Regression': LogisticRegression(), 'KNN': KNeighborsClassifier(), 'Random Forest': RandomForestClassifier(), 'SVM': SVC(), 'Neural Network': MLPClassifier(), 'Linear Regression': LinearRegression()}

In [ ]:
#now iterate through all the models and all the data
# print("Confusion Matrix: ", confusion_matrix(y_test_MCF, y_pred_MCF))
#     print(classification_report(y_test_loop, y_pred))
for key in data:
    for model in models:
        models[model].fit(data[key][0], data[key][2])
        y_pred = models[model].predict(data[key][1])
        print(model, key, accuracy_score(data[key][3], y_pred), classification_report(data[key][3], y_pred))

In [ ]:
#now do this with a grid search to find optimal parameters
params = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [ ]:
#print best parameters
for key in data:
    for model in models:
        grid = GridSearchCV(models[model], params, cv=5)
        grid.fit(data[key][0], data[key][2])
        y_pred = grid.predict(data[key][1])
        print(model, key, accuracy_score(data[key][3], y_pred), classification_report(data[key][3], y_pred))
        print(grid.best_params_)